In [1]:
from r2r import R2RClient
from scraper import Scraper
from splitter import Splitter
from r2r_backend import R2RBackend
from helper.io_helper import iterate_over_files, load_prompt

urls = ["https://en.wikipedia.org/wiki/Bitcoin", "https://en.wikipedia.org/wiki/Dogecoin"]

scraper = Scraper()
splitter = Splitter()
backend = R2RBackend()
backend.health() # Should return OK

2024-10-22 14:25:10,259 - WARNING - langchain_community.utils.user_agent - USER_AGENT environment variable not set, consider setting it to identify your requests.


app=AppConfig(project_name=None) extra_fields={} provider='r2r' default_system_name='default_system' default_task_name='custom_rag' file_path=None


{'response': 'ok'}

In [2]:
# Ingestion
try:
    files = list(iterate_over_files("files"))
    ingestion_resp = backend.ingest_files(files)
    ingestion_resp
except Exception as e:
    pass

2024-10-22 14:11:27,216 - ERROR - r2r_backend - Error during ingestion: Must increment version number before attempting to overwrite document 14b6b10c-c2dd-58f7-ac76-c5ee292df57d. Use the `update_files` endpoint if you are looking to update the existing version.


In [3]:
documents = scraper.fetch_documents(urls)
for document in documents:
    split_document = splitter.split_documents([document])
    chunks = [{"text": chunk.page_content} for chunk in split_document]
    metadata = document.metadata
    try:
        resp = backend.ingest_chunks(chunks, metadata)
        print(resp)
    except Exception as e:
        pass

2024-10-22 14:11:31,872 - ERROR - r2r_backend - Error during chunk ingestion: Document a7aede8d-2dbd-5b1c-8bd1-f28575564b7a was already ingested and is not in a failed state.
2024-10-22 14:11:32,143 - ERROR - r2r_backend - Error during chunk ingestion: Document 621d270e-b632-5402-adb8-dfdffd9bf6d5 was already ingested and is not in a failed state.


In [4]:
docs_metadata = backend.documents_overview()

In [5]:
doc_id = backend.documents_overview()[0]['id']
chunks = backend.document_chunks(doc_id)
#[print(chunk['text']) for chunk in chunks]

In [2]:
query = "What are microservices and how do they work?"
rag_rsp = backend.rag(query)
print(rag_rsp['completion']['choices'][0]['message']['content'])

Microservices are a software development approach that involves breaking down an application into small, independent services that communicate with each other. Each microservice is responsible for a specific business capability or function, and they work together to provide the overall functionality of the application.

Here are some key characteristics of microservices:

1. **Decoupling**: Microservices are designed to be decoupled from each other, meaning that changes to one service do not affect others.
2. **Autonomy**: Each microservice is responsible for its own development, testing, and deployment.
3. **Organized around business capabilities**: Microservices are organized around concrete business capabilities, such as orders, shipping, or payment processing.
4. **Smart endpoints and dumb pipes**: Microservices communicate with each other using simple protocols (dumb pipes) and focus on providing specific functionality (smart endpoints).
5. **Design for failure**: Microservices ar